In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)
DATA_NAME = "x5_sin" # ReCo


In [ ]:
import get_data
df_obs,descriptor_names, target_name = get_data.load(DATA_NAME)
Xraw = df_obs.loc[:, descriptor_names].values
y = df_obs.loc[:, target_name].values


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Xraw)
X = scaler.transform(Xraw)


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
# 計算パラメタ設定
N_SPLITS = 5
SHUFFLE = True
score_function = r2_score
# 性能評価値を保存する変数の定義
train_score_list = []
test_score_list = []
alpha_list = np.logspace(-5, 2, 20)
# hyperparameter loop
for alpha in alpha_list:
    reg = Lasso(alpha=alpha)
    # CV setting
    kf = KFold(n_splits=N_SPLITS, shuffle=SHUFFLE,
               random_state=1)
    # CV loop
    cv_train_score_list = []
    cv_test_score_list = []
    for train, test in kf.split(X):
        Xtrain, ytrain = X[train], y[train]
        Xtest, ytest = X[test], y[test]
        reg.fit(Xtrain, ytrain)
        ytrainp = reg.predict(Xtrain)
        ytestp = reg.predict(Xtest)
        # 評価指標の計算と保存
        trainscore = score_function(ytrain, ytrainp)
        cv_train_score_list.append(trainscore)
        testscore = score_function(ytest, ytestp)
        cv_test_score_list.append(testscore)
    train_score_list.append([np.mean(cv_train_score_list), 
                             np.std(cv_train_score_list), alpha])
    test_score_list.append([np.mean(cv_test_score_list), 
                            np.std(cv_test_score_list), alpha])

In [ ]:
columns_list = ["mean(R2)_train", "std(R2)_train", "alpha"]
df_train_score = pd.DataFrame(train_score_list, columns=columns_list)
columns_list = ["mean(R2)_test", "std(R2)_test", "alpha"]
df_test_score = pd.DataFrame(test_score_list, columns=columns_list)
df_score = df_train_score.merge(df_test_score, on="alpha")

In [ ]:
df_score

In [ ]:
from regression_misc import plot_alpha_yerror
plot_alpha_yerror(df_score)

In [ ]:
imax = np.argmax(df_score["mean(R2)_test"])
alpha_opt = df_score.loc[imax, "alpha"]
print("alpha_opt", alpha_opt)
reg = Lasso(alpha=alpha_opt)
reg.fit(X, y)
print(reg.coef_, reg.intercept_)
yp = reg.predict(X)

In [ ]:
from regression_misc import plot_y_yp

reg = Lasso(alpha=alpha_opt)
kf = KFold(n_splits=N_SPLITS, shuffle=SHUFFLE, random_state=1)
ytest_list = []
ytestp_list = []
# CV loop
for train, test in kf.split(X):
    Xtrain, ytrain = X[train], y[train]
    Xtest, ytest = X[test], y[test]
    # 学習
    reg.fit(Xtrain, ytrain)
    ytrainp = reg.predict(Xtrain)
    ytestp = reg.predict(Xtest)
    ytest_list.append(ytest.tolist())
    ytestp_list.append(ytestp.tolist())
    
# 可視化
plot_y_yp(ytest_list, ytestp_list)


In [ ]:
# ..._new.csvが存在しないDATA_NAMEは以下は動作しない。

df_new,descriptor_names, target_name = get_data.load(DATA_NAME+"_new")

Xraw_new = df_new.loc[:, descriptor_names].values
ynew = df_new.loc[:, target_name].values
Xnew = scaler.transform(Xraw_new)
ynewp = reg.predict(Xnew)


In [ ]:

def plot_y_yp(y, yp, title: str = None, tickfontsize=15, labelfontsize=15):
    """y vs ypを図示する．

    Args:
        y (np.ndarray): 目的変数値
        yp (np.ndarray|list): 目的変数予測値, 1D or 2D.
        title (str, optional): 図のtitle. Defaults to None.
        tickfontsize (int, optional): tick font size. Defaults to 15.
        labelfontsize (int, optional): tick font size. Defaults to 15.
    """
    fig, ax = plt.subplots(figsize=(5, 5))

    # $y^{obs}$ vs $y^{predict}$
    for y1, yp1 in zip(y, yp):
        ax.plot(y1, yp1, "o")

    # 斜め線を引く
    # check y is 1 dim or 2dim
    if len(y[0].shape)>0:
        y = np.concatenate(y)
        yp = np.concatenate(yp)
    yall = np.hstack([y,yp])
    ylim = yall.min(), yall.max()
    ax.plot(ylim, ylim, "--")

    # labelを書く
    ax.set_xlabel("$y_{obs}$", fontsize=labelfontsize)
    ax.set_ylabel("$y_{pred}$", fontsize=labelfontsize)
    if title is not None:
        ax.title(title)
    ax.tick_params(axis="x", labelsize=tickfontsize)
    ax.tick_params(axis="y", labelsize=tickfontsize)
    fig.tight_layout()


plot_y_yp(ynew, ynewp)


In [ ]:
from regression_misc import plot_x1_y
plot_x1_y(X, y, yp, Xnew, ynew, ynewp )

In [ ]:
from sklearn.linear_model import LassoCV
kf = KFold(N_SPLITS, shuffle=True, random_state=1)
reg = LassoCV(cv=kf, alphas=alpha_list)
reg.fit(X, y)
print("alpha", reg.alpha_)
print("coef", reg.coef_)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
result = []
for alpha in alpha_list:
    kf = KFold(N_SPLITS, shuffle=True, random_state=1)
    reg = Lasso(alpha=alpha)
    score_test = cross_val_score(reg, X, y, cv=kf, 
                                 scoring=make_scorer(r2_score))
    result.append([alpha, np.mean(score_test), np.std(score_test)])
df_result = pd.DataFrame(result, 
                         columns=["alpha", "mean(R2)_test", "std(R2)_test"])
plot_alpha_yerror(df_result)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
result_coef = []
for alpha in alpha_list:
    reg = Lasso(alpha=alpha)
    reg.fit(X,y)
    result_coef.append(reg.coef_)
df_coef = pd.DataFrame(result_coef, columns=descriptor_names)
df_alpha = pd.DataFrame({"alpha":alpha_list})
df_coef = df_coef.merge(df_alpha, left_index=True, right_index=True)
df_coef.plot(x="alpha", marker="o", logx=True)
df_coef